<a href="https://colab.research.google.com/github/adithyakini/algo_intraday/blob/master/bhavcopy_nseindia2mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv

base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}

# Before running this script , create a file called bhavcopy_date.txt and write the date from which you want to download EOD data
# Opening file named bhavcopy_date.txt , which keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

for i in range(1,diff.days+1):
    nextdt = lastdt+ relativedelta(days=i)
    if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
      print ("its a saturday or a sunday")
    else:
      print (nextdt.weekday())
      print (nextdt)
      print ("created folders")

      for i in range(7):
          while True:
              try:
                  print ("getting bhavcopy csv")
              except requests.ConnectionError:
                  print('No connection, retrying')
              break



In [ ]:
import requests, zipfile, os, io, pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import mysql.connector
from mysql.connector import Error
import csv

#set the path to where the bhavcopies will be downloaded
base = '/content/drive/My Drive/algotrade/'
today = datetime.today().date()
dmonth={'01':'JAN','02':'FEB','03':'MAR','04':'APR','05':'MAY','06':'JUN','07':'JUL','08':'AUG','09':'SEP','10':'OCT','11':'NOV','12':'DEC'}

# Before running this script , file called bhavcopy_date.txt need to be present in the "base" path.
# Opening file named bhavcopy_date.txt , it keeps track of the last downloaded date.
ltdl = open(base+'bhavcopy_date.txt','r')
lastdt = ltdl.read(10)
ltdl.close()
lastdt = datetime.strptime(lastdt,'%Y-%m-%d')
diff, wr = today-lastdt.date(), ''

#loop through all dates from the last date till today.
for i in range(1,diff.days+1):
    nextdt = lastdt+ relativedelta(days=i)
    #check if the date is a weekend so that we can remove that from the loop , bhavcopies are not available for weekends.
    if (nextdt.weekday() == 5 or nextdt.weekday() == 6):
      print (nextdt.strftime('%Y-%m-%d')+" is a weekend")
    else:
      d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year #extract day , month and year from  the the date
      #if there is no directory already present at the path with the year as a folder then create it
      if not os.path.isdir(base+y):
          os.mkdir(base+y)
          os.mkdir(base+y+'/Index')
          os.mkdir(base+y+'/Futures')
      zpath = base+y+'/'+d+'.zip'
      #try to connect to the nseindia url to download the bhavcopy , 7 times , just incase website does not respond etc.
      for i in range(7):
          while True:
              try:
                  a=requests.get('https://archives.nseindia.com/content/historical/EQUITIES/'+y+'/'+dmonth[m]+'/cm'+d+dmonth[m]+y+'bhav.csv.zip')
              except requests.ConnectionError:
                  print('No connection, retrying')
              break
      #if the connection is successful
      if a.status_code==200:
          dload=open(zpath, 'wb')
          dload.write(a.content)
          dload.close()
          #open the downlaoded bhavcopy and extract it
          z = zipfile.ZipFile(zpath, 'r')
          z.extractall(base+y+'/')
          z.close()
          os.remove(zpath)
          #reading and storing in 2 dictionaries because we need 2 columns from the MTO file deliverable and %deliverable which is not found in the bhavcopy.
          f, deldict = pd.read_csv(base+y+'/cm'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
          f, deldict2 = pd.read_csv(base+y+'/cm'+d+dmonth[m]+y+'bhav.csv'), {}  #reading the raw dl-ed bhav file
          f = f[f['SERIES'] == 'EQ'] #retaining only EQ rows and leaving out bonds,options etc
          deliverable = requests.get('https://archives.nseindia.com/archives/equities/mto/MTO_'+d+m+y+'.DAT').text.splitlines()
          del deliverable[:4]
          
          for i in deliverable:
              c = i.split(',')
              if c[3] == 'EQ' :                
                  deldict[c[2]] = c[5] #building delivarables dict
              if c[3] == 'EQ' :                
                  deldict2[c[2]] = c[6] #building %delivarables dict
      
          dfdel = pd.DataFrame(list(deldict.items()), columns = ['SYMBOL', 'DELIVERABLE'])
          dfdel2 = pd.DataFrame(list(deldict2.items()), columns = ['SYMBOL', '%DELIVERABLE'])
          f = f.merge(dfdel, on='SYMBOL', how='left')      #left merge of delivarables here
          f = f.merge(dfdel2, on='SYMBOL', how='left')      #left merge of delivarables here

          #sometimes nse doesnt give the index file, so the if condition
          indices = requests.get('https://archives.nseindia.com/content/indices/ind_close_all_'+d+m+y+'.csv').content
          if len(indices)>300:
            indx = pd.read_csv(io.StringIO(indices.decode('utf-8'))) #reading content of indices csv and storing in DataFrame using io module
            indx.to_csv(base+y+'/Index/Indices'+ str(nextdt.date())+'.csv', index=False)
            indx[['Index Name', 'Index Date', 'Open Index Value', 'High Index Value', 'Low Index Value', 'Closing Index Value', 'Volume']]
            indx = indx.rename(columns={'Index Name' : 'SYMBOL', 'Index Date' : 'BHAVCOPYDATE', 'Open Index Value' : 'OPEN', 'High Index Value' : 'HIGH', 'Low Index Value' : 'LOW', 'Closing Index Value' : 'CLOSE', 'Volume' : 'TOTTRDQTY'})
            f=f.append(indx, ignore_index=True)
            #write a new csv, bhavcopydate as a column in the csv and get rid of the downloaded file
            f['BHAVCOPYDATE'] = pd.Series(str(nextdt.date().strftime('%Y-%m-%d')) for _ in range(len(f)))
            f = f[['SYMBOL', 'BHAVCOPYDATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'TOTTRDQTY', 'DELIVERABLE','%DELIVERABLE']]
            f.to_csv(base+y+'/'+str(nextdt.date())+'.csv', index=False)
            os.remove(base+y+'/cm'+d+dmonth[m]+y+'bhav.csv')
            #just download and extract futures bhavcopy also
            futures = requests.get('https://archives.nseindia.com/content/historical/DERIVATIVES/'+y+'/'+dmonth[m]+'/fo'+d+dmonth[m]+y+'bhav.csv.zip')
            fo = open(zpath, 'wb')
            fo.write(futures.content)
            fo.close()
            z, wr = zipfile.ZipFile(zpath,'r'), nextdt.date()
            z.extractall(base+y+'/Futures')
            z.close()
            os.remove(zpath)
            print("Downloaded bhavcopy file to "+base+y+'/'+str(nextdt.date())+".csv ...now inserting into DB")

            # connect to MySQL db in https://johnny.heliohost.org:2083/ UN 
            # pip install pip install mysql-connector --target=$nb_path pip install mysql-connector
            # https://pynative.com/python-mysql-database-connection/

            d, m, y = '%02d' % nextdt.day, '%02d' % nextdt.month, '%02d' % nextdt.year

            #check if the path exist and connect to cloud mysql 
            if os.path.exists(base+y):
              try:
                connection = mysql.connector.connect(host='johnny.heliohost.org',
                                                    database='akini_algotrade',
                                                    user='akini',
                                                    password='Drink7up@home')
                if connection.is_connected():
                    db_Info = connection.get_server_info()
                    print("Connected to MySQL Server version ", db_Info)
                    cursor = connection.cursor()
                    cursor.execute("select database();")
                    record = cursor.fetchone()
                    print("You're connected to database: ", record)
                  
                    with open(base+y+'/'+y+'-'+m+'-'+d+'.csv', newline='',  encoding="utf8") as csvfile:
                      csvdata = csv.reader(csvfile)
                      #skip the 1st row as it will be header
                      next(csvdata)

                      for row in csvdata:
                        # Prepare SQL query to INSERT a record into the database.
                        sql = "INSERT INTO bhavcopy (symbol, bhavcopydate, open, high, low, close, tottrdqty, deliverable, deliverable_percent) \
                        VALUES ('%s', '%s','%s', '%s','%s', '%s', '%s', '%s','%s');" % (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8])
                        print(sql)

                        try:
                          # Execute the SQL command
                          cursor.execute(sql)
                          # Commit your changes in the database
                          connection.commit()
                        except Error as e:
                          print("Error while connecting to MySQL", e)
                          connection.rollback()

              except Error as e:
                print("Error while connecting to MySQL", e)
              finally:
                if (connection.is_connected()):
                  cursor.close()
                  connection.close()
                  
print("DONE - All imports complete")


In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

In [2]:
pip install pip install mysql-connector --target=$nb_path pip install mysql-connector

  Using cached https://files.pythonhosted.org/packages/41/cf/e3e6b4d494051c07261cae8c403f0f0d0cedad43d980e5255f2c88fd5edf/install-1.3.3-py3-none-any.whl
Processing /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175/mysql_connector-2.2.9-cp36-cp36m-linux_x86_64.whl
